### In our last model in file "Music_Generation_Train1.ipynb", we got only 82% accuracy. However, in order to generate melodious music, we need at least 90% accuracy. 
### So, we have loaded the weights of last epoch from our previous model into this model and also we have added 2 extra layers of LSTM here with more LSTM units. 
### Here, we are fine-tuning our old layers and we have added more layers. In short, here we are doing "Transfer Learning" from old to new model.

In [1]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

Using TensorFlow backend.


In [2]:
data_directory = "../Data2/"
data_file = "Data_Tunes.txt"
charIndex_json = "char_to_index.json"
model_weights_directory = '../Data2/Model_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [3]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) #155222/16 = 9701
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 9637, 64)  #it denotes number of batches. It runs everytime when
        #new batch is created. We have a total of 151 batches.
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 87)
        for batch_index in range(0, 16):  #it denotes each row in a batch.  
            for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means 
                #each time-step character in a sequence.
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1 #here we have added '1' because the
                #correct label will be the next character in the sequence. So, the next character will be denoted by
                #all_chars[batch_index * batch_chars + start + i] + 1. 
        yield X, Y

In [4]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length), name = "embd_1")) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True, name = "lstm_first"))
    model.add(Dropout(0.2, name = "drp_1"))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    model.load_weights("../Data/Model_Weights/Weights_80.h5", by_name = True)
    
    return model

In [5]:
def training_model(data, epochs = 90):
    #mapping character to index
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) #87
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) #155222
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) #check documentation of train_on_batch here: https://keras.io/models/sequential/
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("../Data2/log.csv", index = False)

In [ ]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    training_model(data)

Number of unique characters in our whole tunes database = 87




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embd_1 (Embedding)           (16, 64, 512)             44544     
_________________________________________________________________
lstm_first (LSTM)            (16, 64, 256)             787456    
_________________________________________________________________
drp_1 (Dropout)              (16, 64, 256)             0         
_________________________________________________________________
lstm_1 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_1 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_2 (LSTM)               

Batch: 55, Loss: 3.260678291320801, Accuracy: 0.1376953125
Batch: 56, Loss: 3.3201704025268555, Accuracy: 0.150390625
Batch: 57, Loss: 3.364957809448242, Accuracy: 0.1318359375
Batch: 58, Loss: 3.2458271980285645, Accuracy: 0.1455078125
Batch: 59, Loss: 3.3630290031433105, Accuracy: 0.138671875
Batch: 60, Loss: 3.2287213802337646, Accuracy: 0.140625
Batch: 61, Loss: 3.244297981262207, Accuracy: 0.1611328125
Batch: 62, Loss: 3.3278794288635254, Accuracy: 0.1513671875
Batch: 63, Loss: 3.277700901031494, Accuracy: 0.1474609375
Batch: 64, Loss: 3.300107955932617, Accuracy: 0.15234375
Batch: 65, Loss: 3.2747254371643066, Accuracy: 0.1630859375
Batch: 66, Loss: 3.2456910610198975, Accuracy: 0.1591796875
Batch: 67, Loss: 3.1462767124176025, Accuracy: 0.185546875
Batch: 68, Loss: 3.192640781402588, Accuracy: 0.1884765625
Batch: 69, Loss: 3.2781474590301514, Accuracy: 0.1640625
Batch: 70, Loss: 3.256969451904297, Accuracy: 0.1669921875
Batch: 71, Loss: 3.2020156383514404, Accuracy: 0.171875
Bat

Batch: 44, Loss: 2.2387852668762207, Accuracy: 0.3740234375
Batch: 45, Loss: 2.2744388580322266, Accuracy: 0.3662109375
Batch: 46, Loss: 2.5781586170196533, Accuracy: 0.302734375
Batch: 47, Loss: 2.683321475982666, Accuracy: 0.2783203125
Batch: 48, Loss: 2.5579769611358643, Accuracy: 0.3134765625
Batch: 49, Loss: 2.4809467792510986, Accuracy: 0.3173828125
Batch: 50, Loss: 2.4808716773986816, Accuracy: 0.3154296875
Batch: 51, Loss: 2.519428253173828, Accuracy: 0.31640625
Batch: 52, Loss: 2.5301332473754883, Accuracy: 0.3017578125
Batch: 53, Loss: 2.318150043487549, Accuracy: 0.3271484375
Batch: 54, Loss: 2.4637017250061035, Accuracy: 0.3193359375
Batch: 55, Loss: 2.338085174560547, Accuracy: 0.3359375
Batch: 56, Loss: 2.4411890506744385, Accuracy: 0.34765625
Batch: 57, Loss: 2.4545516967773438, Accuracy: 0.32421875
Batch: 58, Loss: 2.3963820934295654, Accuracy: 0.3388671875
Batch: 59, Loss: 2.482133388519287, Accuracy: 0.302734375
Batch: 60, Loss: 2.3171069622039795, Accuracy: 0.359375


Batch: 32, Loss: 1.8904950618743896, Accuracy: 0.44140625
Batch: 33, Loss: 2.0300452709198, Accuracy: 0.423828125
Batch: 34, Loss: 2.1040940284729004, Accuracy: 0.4208984375
Batch: 35, Loss: 2.034346103668213, Accuracy: 0.4189453125
Batch: 36, Loss: 2.106020450592041, Accuracy: 0.419921875
Batch: 37, Loss: 2.075193405151367, Accuracy: 0.42578125
Batch: 38, Loss: 1.9712916612625122, Accuracy: 0.4306640625
Batch: 39, Loss: 2.022064208984375, Accuracy: 0.4345703125
Batch: 40, Loss: 2.1565542221069336, Accuracy: 0.4287109375
Batch: 41, Loss: 2.089852809906006, Accuracy: 0.4365234375
Batch: 42, Loss: 1.8232779502868652, Accuracy: 0.4755859375
Batch: 43, Loss: 1.8631235361099243, Accuracy: 0.443359375
Batch: 44, Loss: 1.8316395282745361, Accuracy: 0.4697265625
Batch: 45, Loss: 1.758455514907837, Accuracy: 0.4775390625
Batch: 46, Loss: 2.031264066696167, Accuracy: 0.44140625
Batch: 47, Loss: 2.117203712463379, Accuracy: 0.4091796875
Batch: 48, Loss: 1.9580832719802856, Accuracy: 0.4609375
Bat

Batch: 20, Loss: 1.7794570922851562, Accuracy: 0.5107421875
Batch: 21, Loss: 1.6258368492126465, Accuracy: 0.52734375
Batch: 22, Loss: 1.8232612609863281, Accuracy: 0.4697265625
Batch: 23, Loss: 1.6577234268188477, Accuracy: 0.5068359375
Batch: 24, Loss: 1.7494055032730103, Accuracy: 0.484375
Batch: 25, Loss: 1.6384801864624023, Accuracy: 0.5068359375
Batch: 26, Loss: 1.5785644054412842, Accuracy: 0.5458984375
Batch: 27, Loss: 1.6877998113632202, Accuracy: 0.4951171875
Batch: 28, Loss: 1.7093384265899658, Accuracy: 0.490234375
Batch: 29, Loss: 1.7428278923034668, Accuracy: 0.48828125
Batch: 30, Loss: 1.7487263679504395, Accuracy: 0.509765625
Batch: 31, Loss: 1.735903263092041, Accuracy: 0.5087890625
Batch: 32, Loss: 1.622748613357544, Accuracy: 0.5234375
Batch: 33, Loss: 1.807680606842041, Accuracy: 0.48828125
Batch: 34, Loss: 1.8917003870010376, Accuracy: 0.4619140625
Batch: 35, Loss: 1.784576177597046, Accuracy: 0.4931640625
Batch: 36, Loss: 1.7859129905700684, Accuracy: 0.5087890625

...

Batch: 34, Loss: 0.925922691822052, Accuracy: 0.7060546875
Batch: 35, Loss: 0.7981334924697876, Accuracy: 0.7333984375
Batch: 36, Loss: 0.8600103259086609, Accuracy: 0.728515625
Batch: 37, Loss: 0.7753693461418152, Accuracy: 0.755859375
Batch: 38, Loss: 0.841866135597229, Accuracy: 0.716796875
Batch: 39, Loss: 0.8420273661613464, Accuracy: 0.724609375
Batch: 40, Loss: 0.8204824924468994, Accuracy: 0.75390625
Batch: 41, Loss: 0.7913720607757568, Accuracy: 0.744140625
Batch: 42, Loss: 0.6140881776809692, Accuracy: 0.8017578125
Batch: 43, Loss: 0.8571716547012329, Accuracy: 0.7060546875
Batch: 44, Loss: 0.8276760578155518, Accuracy: 0.73046875
Batch: 45, Loss: 0.7406158447265625, Accuracy: 0.7490234375
Batch: 46, Loss: 0.7751593589782715, Accuracy: 0.7451171875
Batch: 47, Loss: 0.7420812845230103, Accuracy: 0.7578125
Batch: 48, Loss: 0.6762021780014038, Accuracy: 0.7568359375
Batch: 49, Loss: 0.8468033671379089, Accuracy: 0.712890625
Batch: 50, Loss: 0.8291398286819458, Accuracy: 0.730468

Batch: 22, Loss: 0.8169986009597778, Accuracy: 0.744140625
Batch: 23, Loss: 0.8280127048492432, Accuracy: 0.71484375
Batch: 24, Loss: 0.7823216915130615, Accuracy: 0.7314453125
Batch: 25, Loss: 0.7987767457962036, Accuracy: 0.744140625
Batch: 26, Loss: 0.6622346639633179, Accuracy: 0.775390625
Batch: 27, Loss: 0.7413386702537537, Accuracy: 0.74609375
Batch: 28, Loss: 0.7958415746688843, Accuracy: 0.7265625
Batch: 29, Loss: 0.7652220129966736, Accuracy: 0.7470703125
Batch: 30, Loss: 0.710512101650238, Accuracy: 0.7666015625
Batch: 31, Loss: 0.7164819240570068, Accuracy: 0.767578125
Batch: 32, Loss: 0.7201460599899292, Accuracy: 0.7744140625
Batch: 33, Loss: 0.8196096420288086, Accuracy: 0.73046875
Batch: 34, Loss: 0.886203408241272, Accuracy: 0.712890625
Batch: 35, Loss: 0.8034420013427734, Accuracy: 0.732421875
Batch: 36, Loss: 0.8345588445663452, Accuracy: 0.736328125
Batch: 37, Loss: 0.7722823023796082, Accuracy: 0.7490234375
Batch: 38, Loss: 0.821845531463623, Accuracy: 0.716796875


Batch: 9, Loss: 0.7290166616439819, Accuracy: 0.759765625
Batch: 10, Loss: 0.7221686244010925, Accuracy: 0.763671875
Batch: 11, Loss: 0.8753782510757446, Accuracy: 0.7255859375
Batch: 12, Loss: 0.83676677942276, Accuracy: 0.7236328125
Batch: 13, Loss: 0.655545711517334, Accuracy: 0.78125
Batch: 14, Loss: 0.9038029909133911, Accuracy: 0.6962890625
Batch: 15, Loss: 0.7200008630752563, Accuracy: 0.76953125
Batch: 16, Loss: 0.7660614252090454, Accuracy: 0.75
Batch: 17, Loss: 0.8296760320663452, Accuracy: 0.7333984375
Batch: 18, Loss: 0.833115816116333, Accuracy: 0.720703125
Batch: 19, Loss: 0.794957160949707, Accuracy: 0.73828125
Batch: 20, Loss: 0.6991162300109863, Accuracy: 0.77734375
Batch: 21, Loss: 0.7292184233665466, Accuracy: 0.75390625
Batch: 22, Loss: 0.8199682235717773, Accuracy: 0.740234375
Batch: 23, Loss: 0.7838913798332214, Accuracy: 0.736328125
Batch: 24, Loss: 0.7912645936012268, Accuracy: 0.744140625
Batch: 25, Loss: 0.7887858152389526, Accuracy: 0.75
Batch: 26, Loss: 0.66

Batch: 148, Loss: 0.8392685651779175, Accuracy: 0.740234375
Batch: 149, Loss: 0.7357957363128662, Accuracy: 0.7490234375
Batch: 150, Loss: 0.7219581604003906, Accuracy: 0.7626953125
Batch: 151, Loss: 0.6682960987091064, Accuracy: 0.779296875
Epoch 29/90
Batch: 1, Loss: 1.0005409717559814, Accuracy: 0.669921875
Batch: 2, Loss: 0.8733366131782532, Accuracy: 0.6962890625
Batch: 3, Loss: 0.7336618900299072, Accuracy: 0.75390625
Batch: 4, Loss: 0.6594094038009644, Accuracy: 0.779296875
Batch: 5, Loss: 0.7181371450424194, Accuracy: 0.7626953125
Batch: 6, Loss: 0.7658596634864807, Accuracy: 0.74609375
Batch: 7, Loss: 0.8171024322509766, Accuracy: 0.720703125
Batch: 8, Loss: 0.7764894962310791, Accuracy: 0.74609375
Batch: 9, Loss: 0.7486893534660339, Accuracy: 0.751953125
Batch: 10, Loss: 0.72548508644104, Accuracy: 0.759765625
Batch: 11, Loss: 0.8598321676254272, Accuracy: 0.71484375
Batch: 12, Loss: 0.8261217474937439, Accuracy: 0.734375
Batch: 13, Loss: 0.6468499898910522, Accuracy: 0.77636

Batch: 136, Loss: 0.7490136623382568, Accuracy: 0.75390625
Batch: 137, Loss: 0.7442200183868408, Accuracy: 0.7451171875
Batch: 138, Loss: 0.6763197183609009, Accuracy: 0.771484375
Batch: 139, Loss: 0.7260435819625854, Accuracy: 0.7470703125
Batch: 140, Loss: 0.752963662147522, Accuracy: 0.7490234375
Batch: 141, Loss: 0.8009020686149597, Accuracy: 0.7314453125
Batch: 142, Loss: 0.816673994064331, Accuracy: 0.7333984375
Batch: 143, Loss: 0.767019510269165, Accuracy: 0.7607421875
Batch: 144, Loss: 0.7772795557975769, Accuracy: 0.748046875
Batch: 145, Loss: 0.7215788960456848, Accuracy: 0.763671875
Batch: 146, Loss: 0.7598428726196289, Accuracy: 0.7490234375
Batch: 147, Loss: 0.770388126373291, Accuracy: 0.7373046875
Batch: 148, Loss: 0.8439505100250244, Accuracy: 0.7294921875
Batch: 149, Loss: 0.7266820073127747, Accuracy: 0.7607421875
Batch: 150, Loss: 0.7083657383918762, Accuracy: 0.7646484375
Batch: 151, Loss: 0.6592182517051697, Accuracy: 0.7841796875
Epoch 30/90
Batch: 1, Loss: 0.951

In [ ]:
log = pd.read_csv(os.path.join(data_directory, "log.csv"))
log